In [ ]:
# import external packages
import numpy as np
import json
from pathlib import Path

# import supporting pipeline classes (if any)
from utils.img_registration import image_registration, check_reg_prec
from tools.output_handling import load_dict_list_from_json, load_json_experiments_data, NumpyArrayEncoder


# Running across multiple experiments within a main directory
Requires each directory already have all associated json files saved to a single folder.
This can be done by running the python script: `dict_from_directory.py` where the directory containing folders for individual strains is the first parameter, then the strain names (if selecting a few from many) with the `--strains` option and the folder in which to save the resulting json files using the `--output_dir` option.

```
python dict_from_directory.py /pi/david.grunwald-umw/data/yeast_data/ \
--strains BMY820 BMY822 BMY823 \
--output_dir /home/jocelyn.tourtellotte-umw/yeast_json_output/tracking_experiments/
```


In [ ]:
mRNA_tracking_data_list_dict = load_json_experiments_data("/home/jocelyn.tourtellotte-umw/yeast_json_output/tracking_experiments/")

In [ ]:
all_exp_reg_dict_list = {}
for experiment_name in mRNA_tracking_data_list_dict: # for each experiment associated list of dictionaries
    print(experiment_name)
    current_reg_dict_list = []
    for FoV_dict in mRNA_tracking_data_list_dict[experiment_name]: # for each FoV dictionary in that list (dictionaries with paths to all affiliated FoV images)
        try:
            registration_result = image_registration(FoV_dict, False)
            current_reg_dict_list.append(registration_result) # calculate the registration difference etc
        except FileNotFoundError as e:
            print(f"    --> WARNING: File not found for FoV {FoV_dict.get('FoV_id', 'Unknown')}. Skipping. Error: {e}")
    output_path = f"/home/jocelyn.tourtellotte-umw/yeast_json_output/tracking_experiments/registration/reg_result_{experiment_name}.json"
    with open(output_path, 'w') as f:
        json.dump(current_reg_dict_list, f, indent=4, cls=NumpyArrayEncoder)
        
    all_exp_reg_dict_list[experiment_name] = current_reg_dict_list

In [ ]:
output_path = "/home/jocelyn.tourtellotte-umw/yeast_json_output/tracking_experiments/reg_results_summary.json"
with open(output_path, 'w') as f:
    json.dump(reg_results_by_experiment, f, indent=4, cls=NumpyArrayEncoder)

print(f"Successfully saved results to {output_path}")

In [ ]:
for experiment_name in all_exp_reg_dict_list:
    print(experiment_name)

In [ ]:
reg_results_by_experiment = []
for experiment_name in all_exp_reg_dict_list: # for each experiment associated list of dictionaries
    print(experiment_name)
    # is a set of image registration results
    std_diff_vector, reg_prec, FoV_to_retain, is_outlier = check_reg_prec(all_exp_reg_dict_list[experiment_name])
    reg_results_by_experiment.append({'experiment_name':experiment_name,
                                      'std_diff_vector': std_diff_vector,
                                      'reg_precision_mag': reg_prec,
                                      'FoV_to_retain':FoV_to_retain,
                                      'FoV_to_drop': is_outlier})
    print(f"reg-diff sigma vector: {std_diff_vector}")
    print(f"reg-diff precision: {reg_prec}")
    print(f"FoV reg-diff less than 2sigma {len(FoV_to_retain)} of {len(is_outlier) + len(FoV_to_retain)}")
    print(f"or {len(FoV_to_retain) / (len(is_outlier) + len(FoV_to_retain))}")
    
# reg_results_by_experiment

In [ ]:
mRNA_tracking_registration_list_dict = load_json_experiments_data("/home/jocelyn.tourtellotte-umw/yeast_json_output/tracking_experiments/")

In [ ]:
output_path = f"/home/jocelyn.tourtellotte-umw/yeast_json_output/tracking_experiments/registration/reg_results_all_mRNA_tracking.json"
with open(output_path, 'w') as f:
    json.dump(mRNA_tracking_registration_list_dict, f, indent=4, cls=NumpyArrayEncoder)